In [1]:
import re
import requests
import json
from bs4 import BeautifulSoup
from datetime import datetime
import time
import traceback

In [2]:
# ===== ユーティリティ関数群 =====
def _text(s):
    return s.strip().replace("\xa0", " ") if s else ""

def _lower(s):
    return _text(s).lower()

def to_int(s):
    if not s:
        return None
    m = re.search(r"(\d[\d,]*)", str(s))
    return int(m.group(1).replace(",", "")) if m else None

def to_float_percent(s):
    if not s:
        return None
    m = re.search(r"([\d.]+)%", str(s))
    return float(m.group(1)) / 100.0 if m else None

def to_multiplier_int(s):
    if not s:
        return None
    m = re.search(r"×\s*(\d+)", str(s))
    return int(m.group(1)) if m else None

def parse_x_rate(s):
    if not s:
        return None
    m = re.match(r"(\d+)×(\d+)%\s*\(([\d.]+)\)", s)
    if not m:
        return None
    return int(m.group(1)), int(m.group(2)) / 100.0, float(m.group(3))

def safe_set(d, key, val):
    """既に存在するキーは上書きしない"""
    if key not in d and val is not None:
        d[key] = val

def _select_fields(rows, wanted):
    """指定されたフィールドだけ抽出"""
    return [{k: r.get(k) for k in wanted if k in r} for r in rows]

def _extract_headers_and_mask(table):
    """thead か tr.table-headers の th を採用。header-note を除去し、checkbox列は除外"""
    ths = table.select("thead tr th")
    if not ths:
        ths = table.select("tbody tr.table-headers th")
    headers, keep_idx = [], []
    for i, th in enumerate(ths):
        for note in th.select(".header-note"):
            note.decompose()
        cls = " ".join(th.get("class", [])).lower()
        if "checkbox" in cls:
            continue
        label_el = th.select_one("a.table-sort") or th.select_one("span") or th.select_one("a")
        label = (label_el.get_text(strip=True) if label_el else th.get_text(strip=True)) or f"col{i+1}"
        headers.append(label)
        keep_idx.append(i)
    return headers, keep_idx

def _parse_table(table):
    """<table> → [ {header: value, ...}, ... ]（説明列や空行を除外）"""
    headers, keep_idx = _extract_headers_and_mask(table)
    rows = []
    for tr in table.select("tbody tr"):
        if "table-headers" in (tr.get("class") or []):
            continue
        tds = tr.find_all("td")
        if not tds:
            continue
        vals = []
        for j in keep_idx:
            if j < len(tds):
                vals.append(_text(tds[j].get_text()))
        if not any(vals):
            continue
        if headers and "Name" in headers:
            try:
                if not vals[headers.index("Name")]:
                    continue
            except Exception:
                pass
        rows.append(dict(zip(headers, vals)) if headers and len(vals) == len(headers)
        else {f"col{k+1}": v for k, v in enumerate(vals)})
    return headers, rows

def _select_fields(rows, wanted):
    return [{k: r.get(k) for k in wanted} for r in rows]

In [3]:
# ========= メイン関数 =========
def fetch_monster_data(url, base_url):
    headers_req = {
        "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"),
        "Accept-Language": "en-US,en;q=0.9,ja;q=0.8",
    }
    r = requests.get(url, headers=headers_req, timeout=20)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")

    name_part = url.replace(base_url, "").strip("/")
    name = name_part.replace("-", " ").title()
    data = {"name": name}

    # ===== 🧩 冒頭のメインカードを限定抽出 =====
    main_card = soup.select_one("div.card, div.monster-info")
    if not main_card:
        main_card = soup # fallback

    card = {}

    # ===== card-field抽出 =====
    for cf in main_card.select("div.card-field"):
        label_el = cf.select_one(".label")
        if not label_el:
            continue
        key = label_el.get_text(strip=True)
        value_el = cf.select_one(".value")

        # ---- Elemental Vulnerability ----
        if key == "Elemental Vulnerability":
            vul_map = {}
            for node in cf.select(".vulnerabilities > div"):
                a = node.select_one("a")
                elem = a.get_text(strip=True) if a else _text(node.get_text())
                if not elem:
                    continue
                status = "-"
                cls = " ".join(node.get("class", [])).lower()
                title = (node.get("title") or "").lower()
                if "immune" in cls or "immune" in title:
                    status = "Immune"
                elif "resistance" in cls or "resistance" in title:
                    status = "Resistance"
                elif "weakness" in cls or "weakness" in title:
                    status = "Weakness"
                elif "absorb" in cls or "absorb" in title:
                    status = "Absorb"
                vul_map[elem] = status

            # === 🔍 Weakness / Resistance / Absorb のみ抽出して再構成 ===
            filtered = {"Weakness": [], "Resistance": [], "Absorb": []}
            for elem, status in vul_map.items():
                if status in filtered:
                    filtered[status].append(elem)

            # 空でないものだけを出力
            result = {k: v for k, v in filtered.items() if v}

            if result:
                safe_set(data, "ElementalVulnerability", result)
            continue

        # ---- Dropped Items ----
        if key == "Dropped Items":
            drops = []
            if value_el:
                for a in value_el.select("a"):
                    item_name = a.get_text(strip=True)
                    tail = a.next_sibling or ""
                    m = re.search(r"\(([\d.]+)%\)", str(tail))
                    prob = float(m.group(1)) / 100.0 if m else 1.0
                    drops.append({"Item": item_name, "Rate": round(prob, 3)})
            if drops:
                safe_set(data, "Dropped Items", drops)
            continue

        # ---- Stolen Items ----
        if key in {"Stolen Item", "Stolen Items"}:
            stolen_items = []
            if value_el:
                text = value_el.get_text(" ", strip=True)
                parts = re.split(r"\s*(?:,|/| and )\s*", text, maxsplit=0)
                for part in parts:
                    if not part or part in {"-", "–", "—"}:
                        continue
                    m = re.search(r"\(([\d.]+)%\)", part)
                    prob = float(m.group(1)) / 100.0 if m else 1.0
                    name = re.sub(r"\s*\([\d.]+%\)\s*", "", part).strip()
                    if name:
                        stolen_items.append({"Item": name, "Rate": round(prob, 3)})
            if stolen_items:
                safe_set(data, "Stolen Items", stolen_items)
            continue

        # Monster Type
        if key == "Monster Type":
            val = cf.select_one(".value")
            if val:
                txt = _text(val.get_text())

                # 特定の文章パターンを判定
                if "divides into a second monster" in txt.lower():
                    safe_set(data, "Monster Type", "Dividing")
                elif "this monster is undead" in txt.lower():
                    safe_set(data, "Monster Type", "Undead")
                else:
                    safe_set(data, "Monster Type", txt)
            continue

        # Attack Elements（単一・複数対応）
        if key in {"Attack Element", "Attack Elements"}:
            elements = []
            if value_el:
                # aタグが並ぶケースを優先（例: <a>Fire</a>, <a>Poison</a>）
                links = value_el.select("a")
                if links:
                    for a in links:
                        t = a.get_text(strip=True)
                        if t and t not in {"-", "–", "—"}:
                            elements.append(t)
                else:
                    # テキストのみの場合は区切りで分割
                    # 例: "Fire, Ice and Lightning" / "Fire / Ice"
                    text = value_el.get_text(" ", strip=True)
                    parts = re.split(r"\s*(?:,|/| and )\s*", text, maxsplit=0)
                    for part in parts:
                        p = part.strip()
                        if p and p not in {"-", "–", "—"}:
                            elements.append(p)
            
            # 重複除去（順序維持）
            seen = set()
            elements = [e for e in elements if not (e in seen or seen.add(e))]

            if elements:
                # 常に配列で格納（単一でも配列）
                safe_set(data, "Attack Elements", elements)
            continue

        # Status Attack
        if key == "Status Attack":
            val = cf.select_one(".value")
            if val:
                safe_set(data, "Status Attack", _text(val.get_text()))
            continue
        
        # Special Attack Rate
        if key == "Special Attack Rate":
            val = cf.select_one(".value")
            if val:
                rate = _text(val.get_text())
                num = to_float_percent(rate)
                if num is not None:
                    safe_set(data, "SpecialAttackRate", num)
                else:
                    rate
            continue

        # Special Attacks（複数対応）
        if key in {"Special Attack", "Special Attacks"}:
            specials = []
            if value_el:
                # 例: "Blaze (12.5%), Poison Sting (30%) and Petrify (5%)"
                text = value_el.get_text(" ", strip=True)
                parts = re.split(r"\s*(?:,|/| and )\s*", text, maxsplit=0)
                for part in parts:
                    if not part or part in {"-", "–", "—"}:
                        continue
                    # 括弧内の％を抽出（なければ 1.0 = 100%）
                    m = re.search(r"\(([\d.]+)%\)", part)
                    prob = float(m.group(1)) / 100.0 if m else 1.0
                    # 技名から "(xx%)" を除去
                    name = re.sub(r"\s*\([\d.]+%\)\s*", "", part).strip()
                    if name:
                        specials.append({"Attack": name, "Rate": prob})
            if specials:
                safe_set(data, "Special Attacks", specials)
            continue

        # ---- 通常の値 ----
        val_tag = value_el.select_one("a") or value_el
        val = _text(val_tag.get_text())
        if not val or val in {"-", "–", "—"}:
            continue
        safe_set(card, key, val)

    # ===== 基本数値 =====
    safe_set(data, "HP", to_int(card.get("HP")))
    safe_set(data, "Level", to_int(card.get("Level")))
    safe_set(data, "JobLevel", to_int(card.get("Job Level")))
    safe_set(data, "Gil", to_int(card.get("Gil")))
    safe_set(data, "Experience", to_int(card.get("Experience")))
    safe_set(data, "Intelligence", to_int(card.get("Intelligence")))
    safe_set(data, "Mind", to_int(card.get("Mind")))
    safe_set(data, "AttackPower", to_int(card.get("Attack Power")))
    safe_set(data, "Defense", to_int(card.get("Defense")))
    safe_set(data, "MagicDefense", to_int(card.get("Magic Defense")))

    # 10×90% (9.0) 形式
    x = parse_x_rate(card.get("Attacks/Accuracy"))
    if x:
        data["Attack"] = {"Count": x[0], "Accuracy": x[1]}
    x = parse_x_rate(card.get("Evasion"))
    if x:
        data["Evasion"] = {"Count": x[0], "Rate": x[1]}
    x = parse_x_rate(card.get("Magic Resistance"))
    if x:
        data["MagicResistance"] = {"Count": x[0], "Rate": x[1]}

    # ===== data-table (補助テーブル) =====
    for block in soup.select("div.data-table"):
        cap = block.select_one(".table-caption")
        title = _text(cap.get_text()) if cap else ""
        tlow = _lower(title)
        tbl = block.select_one(".data-table-container > table")
        if not tbl:
            continue
        headers, rows = _parse_table(tbl)

        if "status ailment" in tlow and rows:
            # === 元データを辞書化 ===
            row = {h: rows[0].get(h) for h in headers}
            # === Immune のみ抽出して再構成 ===
            immune_elems = [k for k, v in row.items() if _text(v) == "Immune"]
            # 空でなければ出力
            if immune_elems:
                safe_set(data, "StatusAilmentVulnerability", {"Immune": immune_elems})
            continue        
        elif tlow.startswith("spells"):
            wanted = ["Name","Type","Level","Power","Accuracy","Multiplier","Target","Status","Element","Cast By"]
            rows = _select_fields(rows, wanted)
            normalized = []
            for r0 in rows:
                rec = dict(r0)
                rec["Level"] = to_int(rec.get("Level"))
                rec["Power"] = to_int(rec.get("Power"))
                acc = to_float_percent(rec.get("Accuracy"))
                if acc is not None:
                    rec["Accuracy"] = acc
                mul = to_multiplier_int(rec.get("Multiplier"))
                if mul is not None:
                    rec["Multiplier"] = mul
                normalized.append(rec)
            safe_set(data, "Spells", normalized)
        elif tlow.startswith("maps"):
            # Name列だけを抽出して、文字列リストに変換
            wanted = ["Name"]
            rows = _select_fields(rows, wanted)
            # 空でないものだけ抽出し、値だけを取り出す
            names = [r["Name"] for r in rows if r.get("Name")]
            if names:
                safe_set(data, "Maps", names)
            continue
        elif "plot battles" in tlow:
            safe_set(data, "PlotBattles", _select_fields(rows, ["Map","Monsters","Notes"]))

    # ===== ここから出力整形（順序とキー名を指定） =====
    def _nonempty(x):
        return x is not None and x != {} and x != []

    # Stolen Items: {"Item":..., "Rate":...} -> {"Item":..., "StealRate":...}
    stolen_out = []
    for it in data.get("Stolen Items", []) or []:
        if not it.get("Item"):
            continue
        stolen_out.append({
            "Item": it.get("Item"),
            "StealRate": it.get("StealRate", it.get("Rate"))  # 互換: 既存がRateならそのまま移行
        })

    # Dropped Items: {"Item":..., "Rate":...} -> {"Item":..., "DropRate":...}
    dropped_out = []
    for it in data.get("Dropped Items", []) or []:
        if not it.get("Item"):
            continue
        dropped_out.append({
            "Item": it.get("Item"),
            "DropRate": it.get("DropRate", it.get("Rate"))  # 互換
        })

    # Spells: 指定のキーだけに絞る
    spells_out = []
    for sp in data.get("Spells", []) or []:
        spells_out.append({
            "Name": sp.get("Name"),
            "Type": sp.get("Type"),
            "Power": sp.get("Power"),
            "Accuracy": sp.get("Accuracy"),
            "Multiplier": sp.get("Multiplier"),
            "Target": sp.get("Target"),
            "Status": sp.get("Status"),
            "Element": sp.get("Element"),
        })

    # 出力順に沿って Ordered 生成（Py3.7+ は dict で挿入順維持）
    out = {}
    if _nonempty(data.get("name")): out["name"] = data.get("name")
    if _nonempty(data.get("HP")): out["HP"] = data.get("HP")
    if _nonempty(data.get("Level")): out["Level"] = data.get("Level")
    if _nonempty(data.get("JobLevel")): out["JobLevel"] = data.get("JobLevel")
    if _nonempty(data.get("Gil")): out["Gil"] = data.get("Gil")
    if _nonempty(data.get("Experience")): out["Experience"] = data.get("Experience")
    if _nonempty(data.get("Intelligence")): out["Intelligence"] = data.get("Intelligence")
    if _nonempty(data.get("Mind")): out["Mind"] = data.get("Mind")
    if _nonempty(data.get("AttackPower")): out["AttackPower"] = data.get("AttackPower")
    if _nonempty(data.get("Attack")):
        out["Attack"] = {
            "Count": data["Attack"].get("Count") or data["Attack"].get("AttackCount"),
            "Accuracy": data["Attack"].get("Accuracy") or data["Attack"].get("AttackAccuracy"),
        }
    if _nonempty(data.get("Defense")): out["Defense"] = data.get("Defense")
    if _nonempty(data.get("Evasion")):
        out["Evasion"] = {
            "Count": data["Evasion"].get("Count") or data["Evasion"].get("EvadeCount"),
            "Rate": data["Evasion"].get("Rate") or data["Evasion"].get("EvadeRate"),
        }
    if _nonempty(data.get("MagicDefense")): out["MagicDefense"] = data.get("MagicDefense")

    if _nonempty(data.get("MagicResistance")):
        out["MagicResistance"] = {
            "Count": data["MagicResistance"].get("Count") or data["MagicResistance"].get("MagicResistCount"),
            "Rate": data["MagicResistance"].get("Rate") or data["MagicResistance"].get("MagicResistRate"),
        }
    if _nonempty(stolen_out): out["Stolen Items"] = stolen_out
    if _nonempty(dropped_out): out["Dropped Items"] = dropped_out
    if _nonempty(data.get("Monster Type")): out["Monster Type"] = data.get("Monster Type")
    if _nonempty(data.get("ElementalVulnerability")):
        # 既に {"Weakness":[...], "Resistance":[...], "Absorb":[...]} 形式
        out["ElementalVulnerability"] = data.get("ElementalVulnerability")
    if _nonempty(data.get("StatusAilmentVulnerability")):
        # 既に {"Immune":[...]} 形式
        out["StatusAilmentVulnerability"] = data.get("StatusAilmentVulnerability")
    if _nonempty(data.get("Status Attack")):
        out["Status Attack"] = data.get("Status Attack")
    if _nonempty(data.get("SpecialAttackRate")):
        out["SpecialAttackRate"] = data.get("SpecialAttackRate")
    if _nonempty(data.get("Special Attacks")):
        out["Special Attacks"] = data.get("Special Attacks")
    if _nonempty(spells_out): out["Spells"] = spells_out
    if _nonempty(data.get("Maps")): out["Maps"] = data.get("Maps")
    if _nonempty(data.get("PlotBattles")): out["PlotBattles"] = data.get("PlotBattles")

    return out

In [4]:
base_url = "https://guides.gamercorner.net/ffiii/monsters/"
monster_name = "xande"  # ← ここを他のモンスター名に変更するだけで再利用可能
url = base_url + monster_name
json_out = fetch_monster_data(url, base_url)

In [5]:
json_out

{'name': 'Xande',
 'HP': 21000,
 'Level': 113,
 'JobLevel': 97,
 'Intelligence': 72,
 'Mind': 72,
 'AttackPower': 175,
 'Attack': {'Count': 11, 'Accuracy': 0.9},
 'Defense': 11,
 'Evasion': {'Count': 5, 'Rate': 0.3},
 'MagicDefense': 48,
 'MagicResistance': {'Count': 2, 'Rate': 0.4},
 'Stolen Items': [{'Item': 'Potion', 'StealRate': 1.0}],
 'ElementalVulnerability': {'Resistance': ['Lightning',
   'Ice',
   'Fire',
   'Air',
   'Earth',
   'Holy',
   'Dark']},
 'StatusAilmentVulnerability': {'Immune': ['Poison',
   'Blind',
   'Mini',
   'Silence',
   'Toad',
   'Petrification',
   'KO',
   'Confusion',
   'Sleep',
   'Paralysis',
   'Partial Petrification']},
 'SpecialAttackRate': 0.99,
 'Special Attacks': [{'Attack': 'Libra', 'Rate': 0.5},
  {'Attack': 'Flame', 'Rate': 0.25},
  {'Attack': 'Meteor', 'Rate': 0.125},
  {'Attack': 'Quake', 'Rate': 0.125}],
 'Spells': [{'Name': 'Meteor',
   'Type': 'Black',
   'Power': 216,
   'Accuracy': 1.0,
   'Multiplier': 15,
   'Target': 'All Enemie